In [1]:
import dgl
import dgl.function as fn
import torch as th
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph

gcn_msg = fn.copy_src(src='h', out='m')
gcn_reduce = fn.sum(msg='m', out='h')

DGL backend not selected or invalid.  Assuming PyTorch for now.
Using backend: pytorch


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [2]:
class GCNLayer(nn.Module):
    def __init__(self, in_feats, out_feats):
        super(GCNLayer, self).__init__()
        self.linear = nn.Linear(in_feats, out_feats)

    def forward(self, g, feature):
        # Creating a local scope so that all the stored ndata and edata
        # (such as the `'h'` ndata below) are automatically popped out
        # when the scope exits.
        with g.local_scope():
            g.ndata['h'] = feature
            g.update_all(gcn_msg, gcn_reduce)
            h = g.ndata['h']
            return self.linear(h)

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layer1 = GCNLayer(1433, 16)
        self.layer2 = GCNLayer(16, 7)

    def forward(self, g, features):
        x = F.relu(self.layer1(g, features))
        x = self.layer2(g, x)
        return x
net = Net()
print(net)

Net(
  (layer1): GCNLayer(
    (linear): Linear(in_features=1433, out_features=16, bias=True)
  )
  (layer2): GCNLayer(
    (linear): Linear(in_features=16, out_features=7, bias=True)
  )
)


In [5]:
from dgl.data import citation_graph as citegrh
import networkx as nx
def load_cora_data():
    data = citegrh.load_cora()
    features = th.FloatTensor(data.features)
    labels = th.LongTensor(data.labels)
    train_mask = th.BoolTensor(data.train_mask)
    test_mask = th.BoolTensor(data.test_mask)
    g = DGLGraph(data.graph)
    return g, features, labels, train_mask, test_mask


In [6]:
def evaluate(model, g, features, labels, mask):
    model.eval()
    with th.no_grad():
        logits = model(g, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = th.max(logits, dim=1)
        correct = th.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [8]:
import numpy as np
import time
g, features, labels, train_mask, test_mask = load_cora_data()
# Add edges between each node and itself to preserve old node representations
g.add_edges(g.nodes(), g.nodes())
optimizer = th.optim.Adam(net.parameters(), lr=1e-2)
dur = []
for epoch in range(50):
    if epoch >=3:
        t0 = time.time()

    net.train()
    logits = net(g, features)
    logp = F.log_softmax(logits, 1)
    loss = F.nll_loss(logp[train_mask], labels[train_mask])

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch >=3:
        dur.append(time.time() - t0)

    acc = evaluate(net, g, features, labels, test_mask)
    print("Epoch {:05d} | Loss {:.4f} | Test Acc {:.4f} | Time(s) {:.4f}".format(
            epoch, loss.item(), acc, np.mean(dur)))

Loading from cache failed, re-processing.
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.
Epoch 00000 | Loss 1.5313 | Test Acc 0.4960 | Time(s) nan
Epoch 00001 | Loss 1.3935 | Test Acc 0.6100 | Time(s) nan
Epoch 00002 | Loss 1.2663 | Test Acc 0.6540 | Time(s) nan
Epoch 00003 | Loss 1.1493 | Test Acc 0.6750 | Time(s) 0.0105
Epoch 00004 | Loss 1.0443 | Test Acc 0.6980 | Time(s) 0.0104
Epoch 00005 | Loss 0.9481 | Test Acc 0.7090 | Time(s) 0.0104
Epoch 00006 | Loss 0.8594 | Test Acc 0.7370 | Time(s) 0.0104


/home/robert/venv/lib/python3.8/site-packages/dgl/data/utils.py:285: UserWarning: Property dataset.feat will be deprecated, please use g.ndata['feat'] instead.
  warnings.warn('Property {} will be deprecated, please use {} instead.'.format(old, new))
/home/robert/venv/lib/python3.8/site-packages/dgl/data/utils.py:285: UserWarning: Property dataset.label will be deprecated, please use g.ndata['label'] instead.
  warnings.warn('Property {} will be deprecated, please use {} instead.'.format(old, new))
/home/robert/venv/lib/python3.8/site-packages/dgl/data/utils.py:285: UserWarning: Property dataset.train_mask will be deprecated, please use g.ndata['train_mask'] instead.
  warnings.warn('Property {} will be deprecated, please use {} instead.'.format(old, new))
/home/robert/venv/lib/python3.8/site-packages/dgl/data/utils.py:285: UserWarning: Property dataset.test_mask will be deprecated, please use g.ndata['test_mask'] instead.
  warnings.warn('Property {} will be deprecated, please use {} 

Epoch 00007 | Loss 0.7771 | Test Acc 0.7510 | Time(s) 0.0106
Epoch 00008 | Loss 0.7016 | Test Acc 0.7510 | Time(s) 0.0108
Epoch 00009 | Loss 0.6330 | Test Acc 0.7540 | Time(s) 0.0111
Epoch 00010 | Loss 0.5706 | Test Acc 0.7540 | Time(s) 0.0110
Epoch 00011 | Loss 0.5136 | Test Acc 0.7540 | Time(s) 0.0109
Epoch 00012 | Loss 0.4622 | Test Acc 0.7540 | Time(s) 0.0109
Epoch 00013 | Loss 0.4158 | Test Acc 0.7510 | Time(s) 0.0109
Epoch 00014 | Loss 0.3742 | Test Acc 0.7530 | Time(s) 0.0109
Epoch 00015 | Loss 0.3369 | Test Acc 0.7540 | Time(s) 0.0108
Epoch 00016 | Loss 0.3037 | Test Acc 0.7550 | Time(s) 0.0108
Epoch 00017 | Loss 0.2740 | Test Acc 0.7590 | Time(s) 0.0109
Epoch 00018 | Loss 0.2474 | Test Acc 0.7570 | Time(s) 0.0111
Epoch 00019 | Loss 0.2236 | Test Acc 0.7560 | Time(s) 0.0111
Epoch 00020 | Loss 0.2020 | Test Acc 0.7550 | Time(s) 0.0110
Epoch 00021 | Loss 0.1826 | Test Acc 0.7540 | Time(s) 0.0110
Epoch 00022 | Loss 0.1651 | Test Acc 0.7550 | Time(s) 0.0110
Epoch 00023 | Loss 0.149